In [37]:
import numpy as np
import EigenPro2.mnist as mnist
import math
import matplotlib.pyplot as plt
import random
import time
(train_x, train_y), (test_x, test_y) = mnist.load()
aug=40
delta=0.01 #noise level


def K(x,y):
    return math.exp(-(np.linalg.norm(x-y))**2/(10))

Load MNIST dataset.
60000 train samples
10000 test samples


In [38]:
y=8
x=4
time1=time.time()
A = [train_x[train_y==i,:] for i in range(9)]

temp = np.concatenate((A[x], A[y]), axis=0)
rdn_indexes=np.random.randint(len(temp),size=300)

train1=temp[rdn_indexes,:] #no augmentation training set
y1=np.ones(len(A[x]))
y2=np.ones(len(A[y]))*(-1)

ytemp=np.concatenate((y1,y2))
ytrain1=ytemp[rdn_indexes]
    
    

    
P=np.random.normal(0,1,size=(aug,len(train1),784))
temp2=train1+delta*P
temp2=np.array([temp2]).reshape(aug*len(train1), 784)
    
train = temp2 #augmentation training set
 
Ytrain=np.tile(ytrain1,aug)

  
    
B= [test_x[test_y==i,:] for i in range(9)]

test = np.concatenate((B[x],B[y]),axis=0)

m=len(test)
y1=np.ones(len(B[x]))
y2=np.ones(len(B[y]))*(-1)
Ytest=np.concatenate((y1,y2))
time2=time.time()
print('Adding augmentation=',time2-time1)

Adding augmentation= 0.3705451488494873


In [39]:
def Ker(X,Y):
    x=np.sum(np.square(X),axis=1)
    y=np.sum(np.square(Y),axis=1)
    x=x.reshape(len(x),1)
    y=y.reshape(len(y),1)
    temp1=x.repeat(len(y),axis=1)
    temp2=y.repeat(len(x),axis=1)
    squ=temp1+np.transpose(temp2)-2*np.matmul(X,np.transpose(Y))
    mat=np.exp(-squ/10)
    return mat

In [40]:
k1=Ker(train,train)
b=Ker(test,train)


In [41]:
print(np.linalg.norm(train1[200]-train[200]))

0.2880039787527178


In [42]:
z=[0,10**(-9),10**(-8), 10**(-5),10**(-4),10**(-3),10**(-2),10**(-1),10**(0)]
for R in z:

    k2=np.matrix(k1)+R*I2
    w1,v1=np.linalg.eig(k2)
    print(w1)

    k_inv=np.linalg.inv(k2)
    w,v=np.linalg.eig(k_inv)
    print(w)

    Y=Ytrain.reshape(len(Ytrain),1)
    temp1=np.matmul(b,k_inv)
    temp2=np.matmul(temp1,Y)

    y_bar=np.average(Ytest)

    ybar=np.multiply(np.ones(m),y_bar)
    num=0
    den=0
    y_pred1=temp2
    y_pred2=np.ravel(y_pred1)
    te = np.where(y_pred2<0, -np.ones_like(y_pred2), np.ones_like(y_pred2))
    mis=[]
    for i in range(len(Ytest)):
        if np.sign(temp2[i])!=np.sign(Ytest[i]):
            mis.append(i)
        
    num = np.linalg.norm(te - Ytest)**2
    den = np.linalg.norm(ybar-Ytest)**2
    errors=(num/den)
    print('misclassification=',mis)
    print(len(mis))

[8.31774612e+01 7.21666411e+01 5.76555051e+01 ... 1.44620403e-02
 1.44634379e-02 1.44617983e-02]
[133.9786829  128.05806005 125.05116197 ...  60.19566406  60.08751673
  60.00439702]
misclassification= [95, 221, 744, 1010, 1025, 1056, 1077, 1088, 1114, 1153, 1155, 1177, 1384, 1417, 1425, 1466, 1487, 1498, 1624, 1631, 1795, 1801, 1805]
23
[8.31774612e+01 7.21666411e+01 5.76555051e+01 ... 1.45673648e-02
 1.45662223e-02 1.44634389e-02]
[133.97866495 128.05804365 125.05114634 ...  60.19566044  60.08751312
  60.00439342]
misclassification= [95, 221, 744, 1010, 1025, 1056, 1077, 1088, 1114, 1153, 1155, 1177, 1384, 1417, 1425, 1466, 1487, 1498, 1624, 1631, 1795, 1801, 1805]
23
[8.31774612e+01 7.21666411e+01 5.76555051e+01 ... 1.44645063e-02
 1.44636930e-02 1.44634479e-02]
[133.97850339 128.05789606 125.0510056  ...  60.19562783  60.08748063
  60.00436102]
misclassification= [95, 221, 744, 1010, 1025, 1056, 1077, 1088, 1114, 1153, 1155, 1177, 1384, 1417, 1425, 1466, 1487, 1498, 1624, 1631, 1795

In [43]:
z=[0,0.001,0.01,1,10]
for R in z:

    k2=np.matrix(k1)+R*I2
    w1,v1=np.linalg.eig(k2)


    k_inv=np.linalg.inv(k2)
    w,v=np.linalg.eig(k_inv)
    print(w)

    Y=Ytrain.reshape(len(Ytrain),1)
    temp1=np.matmul(b,k_inv)
    temp2=np.matmul(temp1,Y)

    y_pred1=temp2
    y_pred2=np.ravel(y_pred1)
    te = np.where(y_pred2<0, -np.ones_like(y_pred2), np.ones_like(y_pred2))
    num = np.linalg.norm(te - Ytest)**2
    mis=[]
    for i in range(len(Ytest)):
        if np.sign(temp2[i])!=np.sign(Ytest[i]):
            mis.append(i)
        

    print('misclassification=',mis)
    print(len(mis))
    print(num/4)

[133.9786829  128.05806005 125.05116197 ...  60.19566406  60.08751673
  60.00439702]
misclassification= [95, 221, 744, 1010, 1025, 1056, 1077, 1088, 1114, 1153, 1155, 1177, 1384, 1417, 1425, 1466, 1487, 1498, 1624, 1631, 1795, 1801, 1805]
23
22.999999999999996
[118.14920767 113.52080587 111.15153355 ...  57.48942374  57.2981655
  57.29541104]
misclassification= [95, 221, 744, 1010, 1025, 1056, 1077, 1088, 1114, 1153, 1155, 1177, 1384, 1417, 1425, 1466, 1487, 1498, 1624, 1631, 1795, 1801, 1805]
23
22.999999999999996
[3.99358145e-02 1.20210424e-02 1.38548980e-02 ... 3.85058098e+01
 3.85040866e+01 3.85036782e+01]
misclassification= [95, 221, 744, 1010, 1025, 1056, 1077, 1088, 1114, 1153, 1155, 1177, 1384, 1417, 1425, 1466, 1487, 1498, 1624, 1631, 1795, 1801, 1805]
23
22.999999999999996
[0.01187966 0.03841694 0.01366743 ... 0.98519903 0.98519854 0.98519861]
misclassification= [95, 221, 744, 1010, 1025, 1056, 1077, 1088, 1114, 1153, 1155, 1177, 1384, 1417, 1425, 1466, 1487, 1498, 1624, 1631

In [44]:
N2=len(train1)
b_1=np.zeros((m,N2))
for i in range(m):
    for j in range(N2):
        b_1[i][j]=K(test[i],train1[j])
b_1=np.matrix(b_1)


I_2=np.identity(N2)
k_1=np.zeros((N2,N2))
for i in range(N2):
    for j in range(N2):
        k_1[i][j]=K(train1[i],train1[j])

In [45]:
z=[10**(-8), 10**(-5),10**(-3),10**(-2),10**(-1),10**(0),10,20]
for R in z:

    k_2=np.matrix(k_1)+R*I_2
    w1,v1=np.linalg.eig(k_2)


    k_inv2=np.linalg.inv(k_2)
    w,v=np.linalg.eig(k_inv2)


    Y=ytrain1.reshape(len(ytrain1),1)
    temp1=np.matmul(b_1,k_inv2)
    temp2=np.matmul(temp1,Y)

    y_bar=np.average(Ytest)

    ybar=np.multiply(np.ones(m),y_bar)
    num=0
    den=0
    y_pred1=temp2
    y_pred2=np.ravel(y_pred1)
    te = np.where(y_pred2<0, -np.ones_like(y_pred2), np.ones_like(y_pred2))
    mis=[]
    for i in range(len(Ytest)):
        if np.sign(temp2[i])!=np.sign(Ytest[i]):
            mis.append(i)
        
    num = np.linalg.norm(te - Ytest)**2
    den = np.linalg.norm(ybar-Ytest)**2
    errors=(num/den)
    print('misclassification=',mis)
    print(len(mis))

misclassification= [95, 221, 744, 1010, 1025, 1056, 1077, 1088, 1114, 1153, 1155, 1177, 1384, 1417, 1425, 1466, 1487, 1498, 1624, 1631, 1795, 1801, 1805]
23
misclassification= [95, 221, 744, 1010, 1025, 1056, 1077, 1088, 1114, 1153, 1155, 1177, 1384, 1417, 1425, 1466, 1487, 1498, 1624, 1631, 1795, 1801, 1805]
23
misclassification= [95, 221, 744, 1010, 1025, 1056, 1077, 1088, 1114, 1153, 1155, 1177, 1384, 1417, 1425, 1466, 1487, 1498, 1624, 1631, 1795, 1801, 1805]
23
misclassification= [95, 221, 744, 1010, 1025, 1056, 1077, 1088, 1114, 1153, 1155, 1177, 1384, 1417, 1425, 1466, 1487, 1498, 1624, 1631, 1795, 1801, 1805]
23
misclassification= [95, 221, 1010, 1025, 1056, 1077, 1088, 1114, 1153, 1155, 1177, 1384, 1417, 1425, 1466, 1487, 1498, 1624, 1631, 1795, 1801, 1805]
22
misclassification= [95, 221, 1010, 1025, 1056, 1077, 1088, 1114, 1153, 1155, 1177, 1321, 1384, 1417, 1425, 1466, 1487, 1498, 1530, 1624, 1631, 1678, 1795, 1801, 1805, 1943]
26
misclassification= [221, 1010, 1025, 1056, 1